In [1]:
# chargement des données
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import metrics
from scipy.stats import chi2_contingency
import scipy.stats as stats
import os
import math

import warnings
warnings.filterwarnings("ignore")

In [2]:
hackhaton=pd.read_csv("HACKHATON_code_replaced_known_codes.csv",encoding='ISO-8859-1')
hackhaton.drop('Unnamed: 10', axis =1, inplace=True)
hackhaton.drop('Unnamed: 11', axis =1, inplace=True)
hackhaton

,Unnamed: 0.1,Unnamed: 0,ID_CARRIERE,date_debut,date_fin,ETAT,CATEGORIE_EMPLOI,QUALITE,STATUT_AGENT,INDICE_BRUT,EMPLOI_STATUTAIRE,LIBELLE_EMPLOI_GRADE
0,0,0,56,01/01/1999,26/03/1999,1,A,T,T,NaN,NaN,ADJOINT ADMINISTRATIF.........
1,1,1,56,27/03/1999,16/07/1999,1,A,T,T,NaN,NaN,ADJOINT ADMINISTRATIF.........
2,2,2,56,17/07/1999,31/12/1999,1,A,T,T,NaN,NaN,ADJOINT ADMINISTRATIF.........
3,3,3,56,01/01/2000,02/07/2000,1,A,T,T,NaN,NaN,ADJOINT ADMINISTRATIF.........
4,4,4,56,03/07/2000,19/07/2000,1,A,T,T,NaN,NaN,ADJOINT ADMINISTRATIF.........
...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,1048570,1048570,1861885,01/02/2009,28/02/2009,1,A,T,T,303.0,NaN,ADJ PAT 2C
1048571,1048571,1048571,1861885,01/03/2009,31/03/2009,1,A,T,T,303.0,NaN,ADJ PAT 2C
1048572,1048572,1048572,1861885,01/04/2009,08/04/2009,1,A,T,T,303.0,NaN,ADJ PAT 2C
1048573,1048573,1048573,1861885,09/04/2009,19/04/2009,1,A,T,T,303.0,NaN,ADJ PAT 2C


In [3]:
hackhaton2=hackhaton

In [4]:
a=list(hackhaton['EMPLOI_STATUTAIRE'])

len([i for i in a if str(i) == 'nan'])

611149

# Regler la casse et enlever les espaces des libellés

In [5]:
#On enleve aussi les espaces on sait jamais
hackhaton2['LIBELLE_EMPLOI_GRADE']=[(str(i).replace(" ", "")).lower() for i in hackhaton['LIBELLE_EMPLOI_GRADE']]

In [6]:
hackhaton2

,Unnamed: 0.1,Unnamed: 0,ID_CARRIERE,date_debut,date_fin,ETAT,CATEGORIE_EMPLOI,QUALITE,STATUT_AGENT,INDICE_BRUT,EMPLOI_STATUTAIRE,LIBELLE_EMPLOI_GRADE
0,0,0,56,01/01/1999,26/03/1999,1,A,T,T,NaN,NaN,adjointadministratif.........
1,1,1,56,27/03/1999,16/07/1999,1,A,T,T,NaN,NaN,adjointadministratif.........
2,2,2,56,17/07/1999,31/12/1999,1,A,T,T,NaN,NaN,adjointadministratif.........
3,3,3,56,01/01/2000,02/07/2000,1,A,T,T,NaN,NaN,adjointadministratif.........
4,4,4,56,03/07/2000,19/07/2000,1,A,T,T,NaN,NaN,adjointadministratif.........
...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,1048570,1048570,1861885,01/02/2009,28/02/2009,1,A,T,T,303.0,NaN,adjpat2c
1048571,1048571,1048571,1861885,01/03/2009,31/03/2009,1,A,T,T,303.0,NaN,adjpat2c
1048572,1048572,1048572,1861885,01/04/2009,08/04/2009,1,A,T,T,303.0,NaN,adjpat2c
1048573,1048573,1048573,1861885,09/04/2009,19/04/2009,1,A,T,T,303.0,NaN,adjpat2c


# Rajout des codes logiques

## Si un individu est échelon 1 a la date (t), aux dates (t-i) il l'est aussi

environ 20 minutes

In [7]:
#Indice de tous les individus

L=[i for i in list(set(hackhaton2.ID_CARRIERE))]
L = list(set(L))
L=sorted(L)

In [8]:
c=1
for ind in L:
    
    print(c,'/',len(L))
    c=c+1
    df=hackhaton2[hackhaton2.ID_CARRIERE== ind]
    
    if len(df)<2:
        pass

    ## Nombre de ligne avant le premier code
    df = df.reset_index()

    i=0           
    while str(df['EMPLOI_STATUTAIRE'][i]) == 'nan' and (i+1 < len(df)) :
            i=i+1

    df=df.drop([k for k in range (len(df)-1,i,-1)])
    df=df.iloc[::-1]
    df = df.reset_index()
    if len(df)<2:
        pass

    if str(df['EMPLOI_STATUTAIRE'][0])[len(str(df['EMPLOI_STATUTAIRE'][0]))-1]=='1':

        hackhaton2['EMPLOI_STATUTAIRE'][df['index'][len(df)-1]:df['index'][0]] = pd.Series([df['EMPLOI_STATUTAIRE'][0]]*(len(df['EMPLOI_STATUTAIRE'])-1))

1 / 14010
2 / 14010
3 / 14010
4 / 14010
5 / 14010
6 / 14010
7 / 14010
8 / 14010
9 / 14010
10 / 14010
11 / 14010
12 / 14010
13 / 14010
14 / 14010
15 / 14010
16 / 14010
17 / 14010
18 / 14010
19 / 14010
20 / 14010
21 / 14010
22 / 14010
23 / 14010
24 / 14010
25 / 14010
26 / 14010
27 / 14010
28 / 14010
29 / 14010
30 / 14010
31 / 14010
32 / 14010
33 / 14010
34 / 14010
35 / 14010
36 / 14010
37 / 14010
38 / 14010
39 / 14010
40 / 14010
41 / 14010
42 / 14010
43 / 14010
44 / 14010
45 / 14010
46 / 14010
47 / 14010
48 / 14010
49 / 14010
50 / 14010
51 / 14010
52 / 14010
53 / 14010
54 / 14010
55 / 14010
56 / 14010
57 / 14010
58 / 14010
59 / 14010
60 / 14010
61 / 14010
62 / 14010
63 / 14010
64 / 14010
65 / 14010
66 / 14010
67 / 14010
68 / 14010
69 / 14010
70 / 14010
71 / 14010
72 / 14010
73 / 14010
74 / 14010
75 / 14010
76 / 14010
77 / 14010
78 / 14010
79 / 14010
80 / 14010
81 / 14010
82 / 14010
83 / 14010
84 / 14010
85 / 14010
86 / 14010
87 / 14010
88 / 14010
89 / 14010
90 / 14010
91 / 14010
92 / 140


KeyboardInterrupt



In [ ]:
hackhaton=pd.read_csv("HACKHATON_code_replaced_known_codes.csv",encoding='ISO-8859-1')
hackhaton.drop('Unnamed: 10', axis =1, inplace=True)
hackhaton.drop('Unnamed: 11', axis =1, inplace = True)

In [ ]:
#Combien de valeur a-t-on remplacé ? 

c=0     
d=0
for i in hackhaton2.EMPLOI_STATUTAIRE : 
    if str(i)=='nan' :
        c=c+1

for j in hackhaton.EMPLOI_STATUTAIRE : 
    if str(j)=='nan' :
        d=d+1
        

print('il y a ',c,' nan restant maintenant sur ', d,' au départ')  

In [ ]:
hackhaton2.to_csv('hackhaton_logic_replaced.csv')

In [ ]:
hackhaton_logic_replaced=pd.read_csv('hackhaton_logic_replaced.csv',encoding='ISO-8859-1')
hackhaton_logic_replaced.drop('Unnamed: 0', axis =1, inplace=True)

## Correspondances libellés + Un trou entre deux codes identiques 

environ 20 minutes

In [ ]:
hackhaton=hackhaton_logic_replaced
hackhaton

In [ ]:
## TOUS LES TROUS APRES 2011

L1=[]
for i in range (0,1048574) :
    date=hackhaton['date_debut'][i]
    if int(date[6:10])>2010 :
        if str(hackhaton['EMPLOI_STATUTAIRE'][i]) == 'nan':
            L1.append(i)
len(L1)

In [ ]:
## Liste des gens a qui il manque des valeurs apres 2011

L=[hackhaton['ID_CARRIERE'][i] for i in L1]
L = list(set(L))
L=sorted(L)

In [ ]:
hackhaton2=hackhaton

In [ ]:
ind=0
arret=0
compteur = 0

for t in L : 
    #arret=arret+1
    #if arret == 100 :
        #break
    compteur = 1+ compteur
    print(compteur ,'personne sur' , len(L))
        
    df=hackhaton2[hackhaton2.ID_CARRIERE== t]
    
    if len(df)<2:
        pass
    
    ## Nombre de ligne avant 2011
    df = df.reset_index()

    i=0           
    while str(df['EMPLOI_STATUTAIRE'][i]) == 'nan' and (i+1 < len(df)) :
            i=i+1
    
    ##Drop les lignes avant 2011
    
    if i != 0 : #on enleve que si il y a des nan au debut, sinon non   
        df.drop([n for n in range (0,i)], inplace=True)

    if len(df)<2:
        pass

    for k in range (i+1,len(df)+i): 

        if str(df['EMPLOI_STATUTAIRE'][k]) == 'nan' :
            
            #On remplace les nan par correspondance EXACTE eds libelles 
            
            libelle_avant_nan=df['LIBELLE_EMPLOI_GRADE'][k-1]
            libelle_du_nan=df['LIBELLE_EMPLOI_GRADE'][k]
            
            if libelle_avant_nan == libelle_du_nan :

                #si egalité des libellés, on remplace (df['EMPLOI_STATUTAIRE'][i])
                #df.iloc[k,9]=df['EMPLOI_STATUTAIRE'][k-1]
                hackhaton2['EMPLOI_STATUTAIRE'][df['index'][k]]=df['EMPLOI_STATUTAIRE'][k-1]
                df['EMPLOI_STATUTAIRE'][k]=df['EMPLOI_STATUTAIRE'][k-1]



            code_avant_nan=df['EMPLOI_STATUTAIRE'][k-1]

            c=k
            
            ## SI le libelle ne correspond pas, on regarde si il n'a pas changé de poste avant et après
        
            while str(df['EMPLOI_STATUTAIRE'][c]) == 'nan' and c < len(df)+i-1: #sauter les nan entre les deux valeurs 
                c=c+1

            code_après_an = df['EMPLOI_STATUTAIRE'][c]

            if code_avant_nan == code_après_an : #si le code avant et après les nan correspondent, on remplace

                for j in range (0,c-k):

                    df['EMPLOI_STATUTAIRE'][k+j] = code_avant_nan
                    hackhaton2['EMPLOI_STATUTAIRE'][df['index'][k+j]]=code_avant_nan

In [ ]:
#Combien de valeur a-t-on remplacé ? 

c=0
for i in hackhaton2.EMPLOI_STATUTAIRE : 
    if str(i)=='nan' :
        c=c+1
        
print('il y a ',c,' nan restant maintenant sur 611 149  au départ soit ', 611 149 -c ,' codes rajoutés')

In [ ]:
hackhaton2.to_csv('hackhaton_logic_replaced_fin.csv')

In [ ]:
hackhaton=pd.read_csv("hackhaton_logic_replaced_fin.csv",encoding='ISO-8859-1')